In [1]:
%load_ext cython

In [3]:
%%cython

from libc.math cimport sqrt, log, exp, cos
import numpy as np

cdef double loss_gaussian_kernel_two_points(const double[:] X1,
                                       const double[:] X2,
                                       const double[:] k_params):
    """
    Base Gaussian kernel:
        k(X1, X2) = exp(-0.5 * ||X1-X2||_2^2/lam_sqd[j]) 
    Squared-loss Gaussian kernel:
        k_sq(X1, X2) = k(X1[:-product], X2[:-product])^2 
            + k(X1[:-product], X2[:-product]) * <X1[-product:], X2[-product:]>
    between two points X1 and X2
    
    Args:
      X1: array of size d
      X2: array of size d
      k_params: array of (sig_sqd, product), where
        sig_sqd: kernel bandwidth squared
        product: number of dimensions to use for the product kernel 
            (counting backwards from the last dimension)
    """
    
    cdef double sig_sqd = k_params[0]
    cdef long product = <long>(k_params[1]) #1 if use product kernel, 0 otherwise
    cdef long d = X1.shape[0]
    
    cdef long j
    cdef double arg1, arg2, result
    
    # Compute the squared Euclidean distance between X1 and X2
    arg1 = 0
    for j in range(d-product):
        arg1 += (X1[j]-X2[j])**2
    
    # Compute the base squared Gaussian kernel
    result = exp(-0.5 * arg1 / sig_sqd)

    if product:
        # compute linear kernel
        arg2 = 0
        for j in range(d-product, d):
            arg2 += X1[j]*X2[j]
        result = result**2 + result * arg2

    return(result)

X1 = np.array([0,2], dtype=float)
X2 = np.array([0,2], dtype=float)
k_params = np.array([1.0, 1.0], dtype=float)

print('result', loss_gaussian_kernel_two_points(X1, X2, k_params))

Content of stderr:
/Users/ag2435/.cache/ipython/cython/_cython_magic_bb9b4fcc6ed2bf4384e5c81935a7122bf81fe370.c:21976:26: warning: code will never be executed [-Wunreachable-code]
                module = PyImport_ImportModuleLevelObject(
                         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1 warning generated.
ld: warning: duplicate -rpath '/Users/ag2435/anaconda3/envs/goodpoints/lib' ignoredresult 5.0


In [5]:
%%cython

import numpy as np

cdef double loss_gaussian_kernel_one_point(const double[:] X1,
                                      const double[:] k_params):
    """
    Base Gaussian kernel:
        k(X1, X2) = exp(-0.5 * ||X1-X2||_2^2/lam_sqd[j]) 
    Squared-loss Gaussian kernel:
        k_sq(X1, X2) = k(X1[:-product], X2[:-product])^2 
            + k(X1[:-product], X2[:-product]) * <X1[-product:], X2[-product:]>
    between X1 and itself
    
    Args:
      X1: 1D array representing a data point
      k_params: array of (sig_sqd, product), where
        sig_sqd: kernel bandwidth squared
        product: number of dimensions to use for the product kernel 
            (counting backwards from the last dimension)
    """
    cdef long product = <long>(k_params[1]) #1 if use product kernel, 0 otherwise
    cdef long d = X1.shape[0]

    cdef double arg, result 
    
    result = 1

    if product:
        # Compute the linear kernel
        arg = 0
        for j in range(d-product, d):
            arg += X1[j]**2
        result = result**2 + result * arg

    return(result)

X1 = np.array([0,2], dtype=float)
k_params = np.array([1.0, 1.0], dtype=float)
print('result', loss_gaussian_kernel_one_point(X1, k_params))

Content of stderr:
/Users/ag2435/.cache/ipython/cython/_cython_magic_853fcc8910ffd6eea13213159112ba1a07d57348.c:21829:26: warning: code will never be executed [-Wunreachable-code]
                module = PyImport_ImportModuleLevelObject(
                         ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1 warning generated.
ld: warning: duplicate -rpath '/Users/ag2435/anaconda3/envs/goodpoints/lib' ignoredresult 5.0


Goodpoints implementation

In [6]:
from goodpoints.compress import compress_kt
import numpy as np

In [9]:
X = np.ones((10, 2), dtype=float)
kernel_type = b"loss_gaussian"
k_params = np.array([1.0, 1.0], dtype=float)

In [15]:
compress_kt(X, kernel_type, k_params)

array([3, 9])